In [ ]:
%pip install transformers datasets
%pip install accelerate==0.27.2
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
# Importing libraries
from huggingface_hub import login, HfApi
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer, AutoProcessor, AutoFeatureExtractor
from datasets import load_dataset
from torchvision import transforms
import torch
import numpy as np
import gradio as gr
from PIL import Image
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Using CPU")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# Loading the dataset
dataset = load_dataset("cifar10")

In [ ]:
# Lets preprocess the dataset

def preprocess_function(examples):
    transform = transforms.Compose([
        transforms.ToTensor(),  # Convert images to PyTorch tensors
    ])

    examples['pixel_values'] = [transform(img.convert("RGB")) for img in examples['img']]
    return examples

In [ ]:
# Lets apply the preprocessing to the dataset

dataset = dataset.map(preprocess_function, batched=True)

dataset.set_format(type='torch', columns=['pixel_values', 'label'])

In [ ]:
# Lets split the dataset
train_dataset = dataset['train']
test_dataset = dataset['test']

In [ ]:
# Loading the pre-trained ResNet-50 model along with the feature extractor
model = AutoModelForImageClassification.from_pretrained('microsoft/resnet-50', num_labels=10, ignore_mismatched_sizes=True)
model.to(device)

In [ ]:
# Lets define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
)

In [ ]:
# Lets define the compute_metrics function
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = (pred == labels).mean()
    return {"accuracy": accuracy}

In [ ]:
# Perform training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
# Train the model
trainer.train()

In [ ]:
# Evaluate the model
trainer.evaluate()

In [ ]:
# Lets login to the Hugging Face account
login()

In [ ]:
# Lets save the model to the Hugging Face Hub
model_path = "./results/checkpoint-7500"
model = AutoModelForImageClassification.from_pretrained(model_path)